# Senior Isolation Notebook


In [60]:
#Importing modules

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt


#Reading altarum and ipsos csv files, formatting dataframes

altarum_loneliness = pd.read_csv('Data/altarum_loneliness_data.csv')
altarum_visitors = pd.read_csv('Data/altarum_visitors_per_week_data.csv')
ipsos_loneliness = pd.read_csv('Data/ipsos_loneliness_data.csv')

altarum_loneliness = altarum_loneliness.set_index('Unnamed: 0')
altarum_visitors = altarum_visitors.set_index('Unnamed: 0')
ipsos_loneliness = ipsos_loneliness.set_index('Unnamed: 0')


#Reading HRS data from .sas7bdat files, generated from original .da files given by HRS using SAS
hrs_2012_lb = pd.read_sas('Data\h12lb_r.sas7bdat')

#Setting basic styles and formatting for plots
%matplotlib inline
plt.style.use("seaborn")

c:\users\razfyz\vs code projects\senior isolation report\isolation_report_venv\lib\site-packages\pandas\io\sas\sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")


HRS: Loneliness as function of age

Altarum Survey: Loneliness increase after restrictions implemented

Altarum Survey: Visitors before and after restrictions implemented

Ipsos Survey: Loneliness among all Americans (and entire world) from COVID

NSHAP: Correlations of loneliness with depression and aniety among elderly 

HRS: Increased number of physician visits (and hospitalizations) when lonely

In [70]:
#Dropping blank values
hrs_2012_lb = hrs_2012_lb.dropna(axis='index', how='any', subset=['NLB020A', 'NLB020B', 'NLB020C'])

hrs_2012_lb

,HHID,PN,NSUBHH,MSUBHH,NPN_SP,NCSR,NFAMR,NFINR,NLBRTYPE,NLBELIG,...,NLB084U,NLB084V,NLB084W,NLB084X,NLB084Y,NLB084Z,NLB084Z1,NLB085,NVDATE,NVERSION
2,b'010001',b'010',b'0',b'0',NaN,1.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,3.0
12,b'010063',b'010',b'0',b'0',NaN,1.0,1.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,3.0
13,b'010075',b'020',b'0',b'0',NaN,1.0,1.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,3.0
15,b'010097',b'040',b'0',b'0',NaN,1.0,1.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,3.0
21,b'010206',b'010',b'0',b'0',NaN,1.0,1.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20520,b'920639',b'020',b'0',b'0',b'010',1.0,5.0,1.0,4.0,1.0,...,3.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,3.0,3.0
20528,b'920690',b'010',b'0',b'0',NaN,1.0,1.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0
20537,b'920759',b'020',b'0',b'0',b'010',1.0,1.0,5.0,4.0,1.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,4.0,3.0
20544,b'923489',b'020',b'0',b'0',b'010',1.0,1.0,5.0,4.0,1.0,...,3.0,5.0,5.0,4.0,4.0,4.0,3.0,1.0,4.0,3.0
